In [1]:
import os
import requests
import io
import pandas as pd
import pydicom



#see https://stackoverflow.com/questions/15746558/how-to-send-a-multipart-related-with-requests-in-python


In [2]:
from fastai2.data.all import *

In [3]:
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue

In [99]:
# Cell
@patch
def dcmread(fn:Path, force = False): #, force = False):
    "Open a `DICOM` file"
    return pydicom.dcmread(str(fn), force = False)

In [47]:
# Cell
def _cast_dicom_special(x):
    cls = type(x)
    if not cls.__module__.startswith('pydicom'): return x
    if cls.__base__ == object: return x
    return cls.__base__(x)

def _split_elem(res,k,v):
    if not isinstance(v,DcmMultiValue): return
    res[f'Multi{k}'] = 1
    for i,o in enumerate(v): res[f'{k}{"" if i==0 else i}']=o


In [48]:
# Cell
@patch
def as_dict(self:DcmDataset, px_summ=False): #, window=dicom_windows.brain):
    pxdata = (0x7fe0,0x0010)
    vals = [self[o] for o in self.keys() if o != pxdata]
    its = [(v.keyword,v.value) for v in vals]
    res = dict(its)
    res['fname'] = self.filename
    for k,v in its: _split_elem(res,k,v)
    if not px_summ: return res
    stats = 'min','max','mean','std'
#    try:
#        pxs = self.pixel_array
#        for f in stats: res['img_'+f] = getattr(pxs,f)()
#        res['img_pct_window'] = self.pct_in_window(*window)
#    except Exception as e:
#        for f in stats: res['img_'+f] = 0
#        print(res,e)
    for k in res: res[k] = _cast_dicom_special(res[k])
    return res

# Cell
def _dcm2dict(fn, **kwargs): return fn.dcmread().as_dict(**kwargs)


In [49]:
# Cell
@delegates(parallel)
def _from_dicoms(cls, fns, n_workers=0, **kwargs):
    return pd.DataFrame(parallel(_dcm2dict, fns, n_workers=n_workers, **kwargs))
pd.DataFrame.from_dicoms = classmethod(_from_dicoms)

In [170]:
path = Path('C:\\githealth\\dicom-samples\\visus.com\\') #case4\\')

fns = path.ls()
fn = fns[0]

In [155]:
fn.
#for p in path.rglob('*.dcm'):
#    print(p.name)


In [156]:
files = [x for x in path.rglob('*.dcm')]

print(files[:10])

#from pathlib import Path

#for path in Path('C:\\githealth\\dicom-samples\\visus.com\\').rglob('*.dcm'):
#    print(path.name)

[Path('C:/githealth/dicom-samples/visus.com/case1/case1_008.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_010.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_012.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_014.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_016.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_018.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_020.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_022.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_024.dcm'), Path('C:/githealth/dicom-samples/visus.com/case1/case1_026.dcm')]


In [157]:
files = [x for x in path.rglob('*.dcm')]
#files = pydicom.data.get_testdata_files(pattern='*')
for file in files:
    #print(file)
    x = dcmread(file)


In [158]:
df = pd.DataFrame.from_dicoms(files)


In [160]:
df2 = pd.DataFrame()
l=[]

for row in df.iterrows():
    #print(row.dtype())
    l.append(row)
    pass
len(l)

for x,y in l:
    print(type(y))
    df2 = df2.append(y)

    
type(l[0][1])


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

pandas.core.series.Series

In [161]:
len(df2)

96

In [133]:
#for c in df3['fname']:
#    print(c)

In [165]:
df.to_csv('VisusDicom.csv')

In [ ]:
from urllib3.filepost import encode_multipart_formdata, choose_boundary

def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

In [ ]:
fileroot = "C:\\githealth\\dicom-samples\\visus.com\\case4\\"

filename = fileroot + 'case4a_001.dcm'

url = "https://dicom-server-instance.azurewebsites.net/studies"

with open(filepath,'rb') as reader:
    rawfile = reader.read()

files = {'file': ('report.xls', rawfile, 'application/dicom')}

body, content_type = encode_multipart_related(fields = files)
    
headers = {'Accept':'application/dicom+json', "Content-Type":content_type}

r = requests.post(url, body, headers=headers, verify=False)


In [ ]:

print(r.status_code)
print(r.request.headers)
#print(r.request.body)

In [ ]:
def upload_dcm_file(server_url,filepath):
    with open(filepath,'rb') as reader:
        rawfile = reader.read()

    files = {'file': ('report.xls', rawfile, 'application/dicom')}

    body, content_type = encode_multipart_related(fields = files)
    
    headers = {'Accept':'application/dicom+json', "Content-Type":content_type}

    r = requests.post(url, body, headers=headers, verify=False)
    return r

In [ ]:
r = upload_dcm_file(url,'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
print(r.status_code)
print(r.request.headers)

In [ ]:
%timeit
import os
rootdir = "C:\\githealth\\dicom-samples\\"

for subdir, dirs, files in os.walk(rootdir):
    if '.git' in subdir:
        continue
    for file in files:
        if not '.dcm' in file:
            continue
        fname = os.path.join(subdir, file)
        print(fname)
        r = upload_dcm_file(url, fname)
        print(r.status_code)

In [ ]:
!pip install torch torchvision feather-format kornia pyarrow --upgrade

In [ ]:
#!pip install nvidia-ml-py3
#!pip install pydicom kornia opencv-python scikit-image
#!pip install pyarrow

In [ ]:
#!pip install git+https://github.com/fastai/fastai2
!pip install fastai2

In [ ]:
from fastai2 import *
from fastai2.basics import *
from fastai2.medical.imaging import *

In [ ]:
path = Path('../input/rsna-intracranial-hemorrhage-detection/')
path_trn = path/'stage_1_train_images'
fns_trn = path_trn.ls()
fns_trn[:5].attrgot('name')

In [174]:
filename_test = 'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm'
d = pydicom.dcmread(filename_test, force = False)

In [175]:
d.to_json()

'{"00080005": {"Value": ["ISO_IR 100"], "vr": "CS"}, "00080008": {"Value": ["ORIGINAL", "PRIMARY", "M_SE", "M", "SE"], "vr": "CS"}, "00080012": {"Value": ["20051102"], "vr": "DA"}, "00080013": {"Value": ["115144"], "vr": "TM"}, "00080014": {"Value": ["1.3.46.670589.11.5526.5"], "vr": "UI"}, "00080016": {"Value": ["1.2.840.10008.5.1.4.1.1.4"], "vr": "UI"}, "00080018": {"Value": ["1.2.276.0.50.192168001099.8687553.14547392.49"], "vr": "UI"}, "00080020": {"Value": ["20010101"], "vr": "DA"}, "00080021": {"Value": ["20070511"], "vr": "DA"}, "00080022": {"Value": ["20070511"], "vr": "DA"}, "00080023": {"Value": ["20070511"], "vr": "DA"}, "00080030": {"Value": ["114244"], "vr": "TM"}, "00080031": {"Value": ["114244"], "vr": "TM"}, "00080032": {"Value": ["114244"], "vr": "TM"}, "00080033": {"Value": ["114244"], "vr": "TM"}, "00080050": {"Value": ["11788765469991"], "vr": "SH"}, "00080060": {"Value": ["MR"], "vr": "CS"}, "00080070": {"Value": ["Philips Medical Systems"], "vr": "LO"}, "00080080"